In [5]:
import torch
from torch import nn
import torchvision.transforms as T
from sklearn.decomposition import PCA
import sklearn.neighbors
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import umap
import plotly.express as px
import pandas as pd
import os
import h5py 
from torchvision import transforms
import numpy as np
from tqdm import tqdm

In [6]:
import torch
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')

Using cache found in /home/tomwelch/.cache/torch/hub/facebookresearch_dinov2_main
/home/tomwelch/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/tomwelch/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/tomwelch/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [7]:
list(model.modules())[-1].out_features

1000

In [8]:
#path = os.path.join(os.getcwd(), data_name)
path = '/home/tomwelch/Cambridge/neurotransmitter_data'
all_data = []
for date in os.listdir(path):
    if date != '.DS_Store':
        for types in os.listdir(os.path.join(path, date)):
            if types != '.DS_Store':
                for file in os.listdir(os.path.join(path, date, types)):
                    all_data.append([os.path.join(path, date, types, file), types])


In [9]:
latent = []
label = []

batch_size = 36
z_step = 1

print(f"Processing {len(all_data)} files...")

z_ranges = [20,21]
label_list = []

transform = T.Compose([
                T.ToTensor(),
                T.Resize((224, 224)),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

for image_path, label in tqdm(all_data, desc='File', leave=False):
    try:
        with h5py.File(image_path, 'r') as f:
            image = f['volumes/raw'][:]
            
            z_range = range(z_ranges[0], z_ranges[1], z_step)
            
            for j, z_idx in enumerate(z_range):
                img_batch = []
                
                img = image[:, :, z_idx]
                img_rgb = np.stack([img, img, img], axis=2)
                img_tensor = transform(img_rgb).unsqueeze(0)
                img_batch.append(img_tensor)
                
                # Process batch
                if len(img_batch) == batch_size or j == len(z_range) - 1:
                    if img_batch:
                        batch_tensor = torch.cat(img_batch, dim=0)
                        with torch.no_grad():
                            features = model(batch_tensor)
                            latent.append(features)
                            label_list.append(label)
                        img_batch = []
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        continue
print('Done')

Processing 3600 files...


Done


In [13]:
import umap
if latent:
    all_features = torch.cat(latent, dim=0).numpy()

    pca = PCA(n_components=25)
    reduced_features = pca.fit_transform(all_features)
    

    n_neighbor=5
    min_dist=0.01
    n_components=2
    metric='cosine' #cosine
    random_state=10
    
    reducer = umap.UMAP(
        n_neighbors=n_neighbor,
        min_dist=min_dist,
        n_components=n_components,
        metric=metric,
        random_state=random_state
    )
    embedding = reducer.fit_transform(reduced_features)
    
    fig = px.scatter(
        x=embedding[:, 0],
        y=embedding[:, 1],
        color = label_list,
        title = f'UMAP with n_neighbors={n_neighbor}, min_dist={min_dist}, n_components={n_components}, metric={metric},random_state={random_state}',
        width=1500,
        height=1000
    )
    fig.show()
else:
    print("No features were extracted!")
    
    
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=6)
neigh = knn.fit(embedding[:, 0].reshape(-1,1), embedding[:, 1].reshape(-1,1))

/home/tomwelch/anaconda3/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


/home/tomwelch/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() 
                       else 'mps' if torch.backends.mps.is_available() 
                       else "cpu")
        

In [ ]:
type(device)

torch.device

In [ ]:
from PIL import Image
img = Image.open('/home/tomwelch/Cambridge/original/test/y/frarobo_conn_18924918_masks.tif')

T.functional.to_tensor(img).shape

torch.Size([1, 305, 224])